# Config

In [ ]:
import json

In [ ]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="") # input your own APIKey
response = client.chat.completions.create(
    model="glm-4",  
    messages=[
        {"role": "user", "content": "your prompt"},
    ],
)
print(response.choices[0].message)

# Food Entity

In [1]:
from nltk.corpus import wordnet as wn

In [2]:
wn.synsets('food')

[Synset('food.n.01'), Synset('food.n.02'), Synset('food.n.03')]

In [4]:
for f in wn.synsets('food'):
    print(f.definition())

any substance that can be metabolized by an animal to give energy and build tissue
any solid substance (as opposed to liquid) that is used as a source of nourishment
anything that provides mental stimulus for thinking


In [5]:
# choose "any solid substance (as opposed to liquid) that is used as a source of nourishment" as the start root for food entity search


syn=wn.synset('food.n.02')
syn.hyponyms()

[Synset('baked_goods.n.01'),
 Synset('breakfast_food.n.01'),
 Synset('butter.n.01'),
 Synset('cheese.n.01'),
 Synset('chocolate.n.02'),
 Synset('coconut.n.01'),
 Synset('convenience_food.n.01'),
 Synset('dika_bread.n.01'),
 Synset('fish.n.02'),
 Synset('fresh_food.n.01'),
 Synset('health_food.n.01'),
 Synset('junk_food.n.01'),
 Synset('leftovers.n.01'),
 Synset('loaf.n.02'),
 Synset('meat.n.01'),
 Synset('pasta.n.02'),
 Synset('produce.n.01'),
 Synset('seafood.n.01'),
 Synset('slop.n.04'),
 Synset('yogurt.n.01')]

In [6]:
# any substance that can be metabolized by an animal to give energy and build tissue
# set a black list
syn=wn.synset('food.n.01')
syn.hyponyms()

[Synset('beverage.n.01'),
 Synset('chyme.n.01'),
 Synset('comestible.n.01'),
 Synset('comfort_food.n.01'),
 Synset('commissariat.n.01'),
 Synset('culture_medium.n.01'),
 Synset('fare.n.04'),
 Synset('feed.n.01'),
 Synset('foodstuff.n.02'),
 Synset('micronutrient.n.01'),
 Synset('miraculous_food.n.01'),
 Synset('nutriment.n.01'),
 Synset('soul_food.n.01'),
 Synset('water.n.06'),
 Synset('yolk.n.02')]

In [8]:
blacks=['chyme.n.01','comestible.n.01','comfort_food.n.01','commissariat.n.01',
       'culture_medium.n.01','micronutrient.n.01','miraculous_food.n.01',
       'soul_food.n.01']

In [9]:
# Word, POS, SynsetTerm, Gloss, Need
# Depth-First-Search

from queue import LifoQueue

dps=LifoQueue(maxsize=1000000)

#food_concept={} # food_word:ancestor_food_word
dps.put(wn.synset('food.n.01'))
dps.put(wn.synset('food.n.02'))

food_concept={} # food_word:synset_list[]

while dps.qsize()>0:
    cur=dps.get()
    if cur.name() in blacks:
        pass
    else:
        concept=cur.name().split('.')[0]
        if concept not in food_concept:
            food_concept[concept]=[]
        food_concept[concept].append(cur.name()) # add its tag for indexing
        for child in cur.hyponyms():
            dps.put(child)

In [10]:
len(food_concept)

2370

In [ ]:
with open("Data/food_concept_v2.json", "w") as outfile:
    json.dump(food_concept, outfile)

#with open("Data/food_concept.json", "r") as infile:
#    food_concept=json.load(infile)

## Different Senses

    If a word has several senses, among which both food related and unrelated exist, badcases would arise in real applications. For example, "you have a glass cup" is related to food if "cup" takes the sense of "a punch served in a pitcher instead of a punch bowl" provided in WordNet. But in that sentence, "cup" is more likely to bear the sense of "a small open container usually used for drinking; usually has a handle".
    
    We take this solution: remove words with food unrelated senses to keep a high precision with allowing a relatively low recall.

In [11]:
len(food_concept)

2370

In [12]:
food_concept_diff_senses={}
for x,y in  food_concept.items():
    max_num=0
    for yy in y:
        _id=int(yy.split('.')[-1])
        if _id>max_num:
            max_num=_id
    if max_num>1:
        food_concept_diff_senses[x]=y
        
"""    print(x)
    print('|'.join(y))
    print('---------------')
    cnt+=1
    if cnt>20:
        break"""

"    print(x)\n    print('|'.join(y))\n    print('---------------')\n    cnt+=1\n    if cnt>20:\n        break"

In [13]:
len(food_concept_diff_senses)

579

In [ ]:
for x,y in  food_concept_diff_senses.items():
    print(x)
    print('|'.join(y))
    print('---------------')

In [14]:
food_concept_diff_senses2={}
for x,y in  food_concept.items():
    max_num=0
    _ids=[]
    for yy in y:
        _id=int(yy.split('.')[-1])
        _ids.append(_id)
        if _id>max_num:
            max_num=_id
    if max_num>1 and len(_ids)!=max_num:
        food_concept_diff_senses2[x]=y
        
len(food_concept_diff_senses2)

528

In [15]:
food_concept_single_senses={}
for x,y in food_concept.items():
    if x not in food_concept_diff_senses2:
        food_concept_single_senses[x]=y

len(food_concept_single_senses)

1842

In [ ]:
with open("Data/food_concept_v2_single_sense.json", "w") as outfile:
    json.dump(food_concept_single_senses, outfile)

# Emotion

In 1987, Shaver and his colleagues published Emotion Knowledge: Further Exploration of a Prototype Approach where they outlined a three-level hierarchy of emotions and categorized words within that hierarchy. In the paper, they identified six primary emotions: Anger, Fear, Joy, Love, Sadness, and Surprise – And they identified 25 secondary emotions and 135 words that represent more descriptive tertiary emotions.

@article{shaver1987emotion,
  title={Emotion knowledge: further exploration of a prototype approach.},
  author={Shaver, Phillip and Schwartz, Judith and Kirson, Donald and O'connor, Cary},
  journal={Journal of personality and social psychology},
  volume={52},
  number={6},
  pages={1061},
  year={1987},
  publisher={American Psychological Association}
}

https://www.centervention.com/list-of-emotions-135-words-that-express-feelings/

## Noun

In [373]:
# noun seed

emotion={}
emotion['Joy']={}
emotion['Love']={}
emotion['Anger']={}
emotion['Fear']={}
emotion['Sadness']={}
emotion['Surprise']={}

emotion['Joy']['Cheerfulness']=['amusement', 'bliss', 'cheerfulness', 'delight', 'ecstasy', 'elation', 
                                'enjoyment', 'euphoria', 'gaiety', 'gladness', 'glee', 'happiness', 
                                'jolliness', 'joviality', 'joy', 'jubilation', 'satisfaction']
emotion['Joy']['Contentment']=['contentment', 'pleasure']
emotion['Joy']['Enthrallment']=['enthrallment', 'rapture']
emotion['Joy']['Optimism']=['eagerness', 'hope', 'optimism']
emotion['Joy']['Pride']=['pride', 'triumph']
emotion['Joy']['Relief']=['relief']
emotion['Joy']['Zest']=['enthusiasm', 'excitement', 'exhilaration', 'thrill', 'zeal', 'zest']


emotion['Love']['Affection']=['adoration', 'affection', 'attraction', 'caring', 'compassion', 'fondness', 
                              'liking', 'love', 'sentimentality', 'tenderness']
emotion['Love']['Longing']=['longing']
emotion['Love']['Lust']=['arousal', 'desire', 'infatuation', 'lust', 'passion']


emotion['Anger']['Disgust']=['contempt', 'disgust', 'revulsion']
emotion['Anger']['Envy']=['envy', 'jealousy']
emotion['Anger']['Exasperation']=['exasperation', 'frustration']
emotion['Anger']['Irritation']=['aggravation', 'agitation', 'annoyance', 'grouchiness', 'grumpiness', 'irritation']
emotion['Anger']['Rage']=['anger', 'bitterness', 'dislike', 'ferocity', 'fury', 'hate', 'hostility', 'loathing', 'outrage', 'rage', 'resentment', 'scorn', 'spite', 'vengefulness', 'wrath']
emotion['Anger']['Torment']=['torment']


emotion['Fear']['Horror']=['alarm', 'fear', 'fright', 'horror', 'hysteria', 'mortification', 'panic', 'shock', 'terror']
emotion['Fear']['Nervousness']=['anxiety', 'apprehension', 'distress', 'dread', 'nervousness', 'tenseness', 'uneasiness', 'worry']


emotion['Sadness']['Disappointment']=['disappointment', 'dismay', 'displeasure']
emotion['Sadness']['Neglect']=['alienation', 'defeat', 'dejection', 'embarrassment', 'homesickness', 'humiliation', 'insecurity', 'isolation', 'insult', 'loneliness', 'neglect', 'rejection']
emotion['Sadness']['Sadness']=['Depression', 'despair', 'gloom', 'glumness', 'grief', 'hopelessness', 'melancholy', 'misery', 'sadness', 'sorrow', 'unhappiness', 'woe']
emotion['Sadness']['Shame']=['guilt', 'regret', 'remorse', 'shame']
emotion['Sadness']['Suffering']=['agony', 'anguish', 'hurt', 'suffering']
emotion['Sadness']['Sympathy']=['pity', 'sympathy']


emotion['Surprise']['Surprise']=['amazement', 'astonishment', 'surprise']



In [374]:
with open("Data/emo_noun_seed.json", "w") as outfile:
    json.dump(emotion, outfile)

"""with open("Data/emo_noun_seed.json", "r") as infile:
    emotion=json.load(infile)"""

'with open("Data/emo_noun_seed.json", "r") as infile:\n    emotion=json.load(infile)'

## Adj

In [ ]:
# adj seed

emotion={}
emotion['Joy']={}
emotion['Love']={}
emotion['Anger']={}
emotion['Fear']={}
emotion['Sadness']={}
emotion['Surprise']={}

emotion['Joy']['Cheerfulness']=['amusing',  'cheerful', 'delightful', 
                
                                'euphoriant', 'glad', 'gleeful', 'happy', 
                                'jolly', 'jovial', 'joyful']
emotion['Joy']['Contentment']=['content', 'pleased']
emotion['Joy']['Enthrallment']=['enthralling', 'raptured']
emotion['Joy']['Optimism']=['eager', 'hopeful', 'optimistic']
emotion['Joy']['Pride']=['proud', 'triumphant']
emotion['Joy']['Relief']=['relieved']
emotion['Joy']['Zest']=['enthusiastic', 'excited', 'exhilarated', 'thrilled',
                        'zealous', 'zesty']


emotion['Love']['Affection']=['adorable', 'affectionate', 'attractive', 
                              'caring', 'compassionate', 'fond', 
                              'lovely',  'tender']
emotion['Love']['Longing']=['longing']
emotion['Love']['Lust']=['desired', 'infatuated', 'lusty', 'passionate']


emotion['Anger']['Disgust']=['contemptuous', 'disgusting', 'revulsed']
emotion['Anger']['Envy']=['envious', 'jealous']
emotion['Anger']['Exasperation']=['exasperated', 'frustrated']
emotion['Anger']['Irritation']=['aggravated', 'agitated', 'grouchy', 
                                'grumpy', 'irritated']
emotion['Anger']['Rage']=['angry', 'bitter', 'disliked', 'ferocious', 
                          'furious', 'hateful', 'hostile', 'loathing', 
                          'outrageous', 'rageful', 'resentful', 'scornful',
                          'spiteful', 'vengeful', 'wrathful']


emotion['Fear']['Horror']=['alarming', 'fearful', 'frightful','mortified',
                           'panic', 'shocked']
emotion['Fear']['Nervousness']=['anxious', 'apprehensive', 'distressful',
                                'dreadful', 'nervous', 'tense', 'uneasy', 
                                'worrisome']


emotion['Sadness']['Disappointment']=['disappointed', 'displeased']
emotion['Sadness']['Neglect']=['alienated', 'dejected', 'embarrassed', 
                               'homesick', 'insecure', 'isolated', 
                               'insulted', 'lonely', 'neglectful',]
emotion['Sadness']['Sadness']=['depressive', 'despairing', 'gloomy', 
                               'glum', 'grim', 'hopeless', 'melancholy', 
                               'miserable', 'sad', 'sorrowful', 'unhappy', 
                               'woeful']
emotion['Sadness']['Shame']=['guilty', 'regretful', 'remorseful', 'shameful']
emotion['Sadness']['Suffering']=['agonizing', 'anguished', 'hurtful']
emotion['Sadness']['Sympathy']=['pitying', 'sympathetic']


emotion['Surprise']['Surprise']=['amazed', 'astonishing', 'surprised']

with open("Data/emo_adj_seed.json", "w") as outfile:
    json.dump(emotion, outfile)

In [ ]:
# adj extension
# also add past participle and present participle of extended verbs



## Verb

In [27]:
# verb seed

# we try to only keep emotion related glosses in the extended vocabulary. In particular, we remove reasoning-related
# glosses, like "reject". One difficult is some actions are strongly related to emotion or these actions occur to express
# some kind of emotion, like mollycoddle. In this case, we keep these actions in the extended vocabulary.
    

emotion={}
emotion['Joy']={}
emotion['Love']={}
emotion['Anger']={}
emotion['Fear']={}
emotion['Sadness']={}
emotion['Surprise']={}

emotion['Joy']['Cheerfulness']=['amuse', 'cheer', 'delight', 'elate', 
                                'enjoy', 'glad', 'exult',  
                                'jolly', 'joy', 'jubilate', 'satisfy']
emotion['Joy']['Contentment']=['please']
emotion['Joy']['Enthrallment']=['enthrall', 'captivate']
emotion['Joy']['Optimism']=['yearn', 'hope', 'optimize']
emotion['Joy']['Pride']=['pride', 'succeed']
emotion['Joy']['Relief']=['relieve']
emotion['Joy']['Zest']=['enthuse', 'excite', 'exhilarate', 'thrill', 'zest']


emotion['Love']['Affection']=['adore', 'attract', 'care', 'like', 'love']
emotion['Love']['Longing']=['long']
emotion['Love']['Lust']=['arouse', 'desire', 'infatuate', 'lust']


emotion['Anger']['Disgust']=['disgust', 'revulse']
emotion['Anger']['Envy']=['envy']
emotion['Anger']['Exasperation']=['exasperate', 'frustrate']
emotion['Anger']['Irritation']=['aggravate', 'agitate', 'annoy', 'grouch', 'grump', 'irritate']
emotion['Anger']['Rage']=['anger', 'dislike', 'hate',  'loathe', 'outrage', 'rage', 'resent', 'scorn', 'spite',
                          'venge']
emotion['Anger']['Torment']=['torment']


emotion['Fear']['Horror']=['alarm', 'fear', 'fright', 'panic', 'shock', 'terrify']
emotion['Fear']['Nervousness']=['distress', 'dread', 'tense', 'worry']


emotion['Sadness']['Disappointment']=['disappoint', 'dismay', 'displease']
emotion['Sadness']['Neglect']=['alienate', 'defeat', 'deject', 'embarrass', 'homesickness', 'humiliate', 'isolate', 'insult', 'neglect', 'reject']
emotion['Sadness']['Sadness']=['depress', 'despair', 'gloom', 'sorrow']
emotion['Sadness']['Shame']=['regret', 'shame']
emotion['Sadness']['Suffering']=['agonize', 'anguish', 'hurt', 'suffer']
emotion['Sadness']['Sympathy']=['pity', 'sympathize']


emotion['Surprise']['Surprise']=['amaze', 'astonish', 'surprise']



In [28]:
with open("Data/emo_verb_seed.json", "w") as outfile:
    json.dump(emotion, outfile)

In [ ]:
# verb extended

extension={}
extension['amuse']=['entertain','regale','distract','delight','appease','divert','occupy','solace','please','busy','engross','enchant','interest','captivate','engage','fascinate','involve','console','charm','enthral','disport','comfort','beguile','immerse','enthrall','intrigue','bewitch','gratify','hypnotize','placate','spoil','humor','soothe','mesmerize','pleasure','indulge','pamper','pacify','absorb','mollify','coddle','oblige','content','propitiate','conciliate','grip','mollycoddle']




##  EmoAnomalyDetect

    Filter out badcases of emotion words by LLM.

### Positive

In [ ]:
def pos_bad_check(resp):
    for s in re.split('\.|,',resp): # sentence level
        s=s.replace('\n','').split(' ') # bag of words
        if 'False' in s:
            return True

        if 'not' in s and 'positive' in s:
            return True

    return False


        
def neg_bad_check(resp):
    for s in re.split('\.|,',resp): # sentence level
        s=s.replace('\n','').split(' ') # bag of words
        if 'False' in s:
            return True

        if 'not' in s and 'negative' in s:
            return True

    return False

In [ ]:
with open('Data/emo_adj_new.json', "r") as infile:
    emo_adj = json.load(infile)
with open('Data/emo_verb_new.json', "r") as infile:
    emo_verb = json.load(infile)

emo_adj_pos, emo_adj_neg=polarize_emo(emo_adj)
emo_verb_pos, emo_verb_neg=polarize_emo(emo_verb)

emo_pos,emo_neg = {**emo_adj_pos, **emo_verb_pos}, {**emo_adj_neg, **emo_verb_neg}

In [ ]:
res={}
cnt=0
for w, _ in tqdm.tqdm(emo_pos.items()):
    
    prompt = """
    Judge if the INPUT word is describing positive emotion . Answer in English. Explain your reasoning then state the answer. Return =True= or =False=.
    INPUT: %s.
    OUTPUT:
    """ % (w)
    req = {
        "model": model,
        "parameters": params,
        "messages": [
            {
                "role": ChatRole.USER,
                "content": prompt
            }
        ]
    }
    
    resp=test_chat(maas, req)

    res[w]=resp
    
    cnt+=1
    #if cnt>30:
    #   break
    #if cnt in (1000,2000,3000,4000,5000):
    #    joblib.dump(res,'Data/tmp_terrible_exp_adj_part1_%d'%cnt)
        
    

In [ ]:
pos_bad2=[w for w,_ in res.items() if pos_bad_check(_)]

len(pos_bad2)

In [ ]:
with open(pycharm_dir+'/Data/emo_pos_badcases.json', 'w') as outfile:
    json.dump(pos_bad2, outfile)

### Negative

In [ ]:
res2={}
cnt=0
for w, _ in tqdm.tqdm(emo_neg.items()):
    
    prompt = """
    Judge if the INPUT word is describing negative emotion . Answer in English. Explain your reasoning then state the answer. Return =True= or =False=.
    INPUT: %s.
    OUTPUT:
    """ % (w)
    req = {
        "model": model,
        "parameters": params,
        "messages": [
            {
                "role": ChatRole.USER,
                "content": prompt
            }
        ]
    }
    
    resp=test_chat(maas, req)

    res2[w]=resp
    
    cnt+=1
    #if cnt>30:
    #   break
    #if cnt in (1000,2000,3000,4000,5000):
    #    joblib.dump(res,'Data/tmp_terrible_exp_adj_part1_%d'%cnt)
        
    

In [ ]:
neg_bad2=[w for w,_ in res2.items() if neg_bad_check(_)]

len(neg_bad2)

In [ ]:
with open(pycharm_dir+'/Data/emo_neg_badcases.json', 'w') as outfile:
    json.dump(neg_bad2, outfile)

# Feeling

## SentiWordNet

In [ ]:
f = open('source/SentiWordNet_3.0.0.txt',"r") 
lines = f.readlines()

def parser(x):
    xx=x.split('\t')
    xx[5]=xx[5].replace('\n','').replace('\\','').replace('"','') # gloss
    return xx

_net=[]
for line in lines:
    if line[0]=='#':
        continue
    _net.append(parser(line))

net=pd.DataFrame()
net['POS'],net['ID'],net['PosScore'],net['NegScore'],net['SynsetTerms'],net['Gloss']=list(zip(*(_net)))

net['NegScore']=pd.to_numeric(net['NegScore'])
net['PosScore']=pd.to_numeric(net['PosScore'])

net.to_csv('source/SentiWordNet3.csv')


## Round1

In [ ]:
sentinet=pd.read_csv('source/SentiWordNet3.csv')

In [ ]:
res={}
cnt=0
for st, gloss in tqdm.tqdm(zip(sentinet.loc[(sentinet.POS == 'a') & (sentinet.NegScore > 0.33), 'SynsetTerms'],sentinet.loc[(sentinet.POS == 'a') & (sentinet.NegScore > 0.33), 'Gloss'])):
    
    prompt = """
    Judge if the INPUT is describing terrible experience. Answer in English. Explain your reasoning then state the answer. Return =True= or =False=.
    INPUT: %s.
    OUTPUT:
    """ % (gloss)
    
    response = client.chat.completions.create(
    model="glm-4",  
    messages=[
        {"role": "user", "content": "%s"%prompt},
    ],
)
    
    res[st]={}
    res[st]['gloss']=gloss
    res[st]['answer']=response.choices[0].message
    
    cnt+=1
    #if cnt>100:
    #    break
    if cnt in (1000,2000,3000,4000,5000):
        joblib.dump(res,'Data/tmp_terrible_exp_adj_part1_%d'%cnt)
        
    

In [ ]:
neg_exp_synset={}
for st, info in res.items():
    if isinstance(info['answer'],str) and 'True' in info['answer']:
        neg_exp_synset[st]=info['gloss']

joblib.dump(neg_exp_synset,'Data/neg_exp_synset.p1')

## Round2

    To further improve acc, use llm to directly filter out badcases at word level.

In [ ]:
neg_adj = {}
for st,gloss in neg_exp_synset.items():
    for x in st.split(' '):
        neg_adj[x.split('#')[0]] = 1

In [ ]:
res2={}
cnt=0
for w, _ in tqdm.tqdm(neg_adj.items()):
    prompt = """
    Judge if the INPUT word is describing terrible experience. Answer in English. Explain your reasoning then state the answer. Return =True= or =False=.
    INPUT: %s.
    OUTPUT:
    """ % (w)
    
    response = client.chat.completions.create(
    model="glm-4",  
    messages=[
        {"role": "user", "content": "%s"%prompt},
    ],
    )
    
    cnt+=1
    #if cnt>100:
    #    break
    if cnt in (500,1000,1500,2000,2500):
        joblib.dump(res2,'Data/tmp_terrible_exp_adj_part_%d.v2'%cnt)
        
    res2[w]=response.choices[0].message
    
joblib.dump(res2,'Data/tmp_terrible_exp_adj_part_all.v2')

In [ ]:
neg_adj={}
for w, resp in res2.items():
    if isinstance(resp,str) and 'True' in resp:
        neg_adj[w]=1

with open('Data/neg_exp_adj.json', 'w') as outfile:
    json.dump(neg_adj, outfile)

    As currently we have no way to decide which sense of a word should be used, if we use sense text from wordnet as input of LLM, noise would be introduced. In contrast, directly inputting word text gives better performance, as LLM output yes or no based on common sense knowledge, which more complies with human's judgements. 

# ActionType

In [ ]:
noun,verb,adj,adv=joblib.load('source/wordset')

In [ ]:
res={}
cnt=0
for w in tqdm.tqdm(verb):
    
    prompt = """
    Mental: a mental action happens inside human beings and isn't visible. You engage in covert behaviour when you think since no one can see you thinking.

    Physical: a physical action is a behaviour that's visible and happens outside of human beings. Examples of overt behaviour include eating or drinking something and taking part in sports, such as football or riding a bicycle.

    Social: social behavior accounts for actions directed at others. It is concerned with the considerable influence of social interaction and culture, as well as ethics, interpersonal relationships, politics, and conflict.

    Judge the class that the INPUT action belongs to. Answer in English. Explain your reasoning then state the answer. Return =Mental= or =Physical= or =Social=.
    INPUT: %s.
    OUTPUT:
    """ % (w)
    
    response = client.chat.completions.create(
    model="glm-4",  
    messages=[
        {"role": "user", "content": "%s"%prompt},
    ],
    )
    

    res[w]=response.choices[0].message
    
    cnt+=1
    
    #if cnt>30:
    #    break
    if cnt in (10,1000,2000,3000,4000,5000):
        joblib.dump(res,'Data/action_type_%d'%cnt)
        
joblib.dump(res,'Data/action_type')    

In [ ]:
mental={}
physical={}
social={}
unknown={}

for w, resp in res.items():
    # a word might be in multiple categories
    if isinstance(resp,str):
        if 'Mental' in resp or 'mental' in resp or 'not visible' in resp or "isn't visible" in resp or "aren't visible" in resp or \
            'happens inside' in resp or 'happen inside' in resp:
            mental[w]=resp
        if 'Physical' in resp or 'physical' in resp or 'happens outside' in resp or 'happen outside' in resp:
            physical[w]=resp
        if 'Social' in resp or 'social' in resp:
            social[w]=resp

for w, resp in res.items():
    if not (w in mental or w in physical or w in social):
        unknown[w]=resp

print("Mental: %d"%len(mental))
print("Physical: %d"%len(physical))
print("Social: %d"%len(social))
print("Unknown: %d"%len(unknown))

joblib.dump((mental,physical,social,unknown), 'Data/action_type_set')

In [ ]:
action_type={}
action_type['mental']=mental
action_type['physical']=physical
action_type['social']=social
action_type['unknown']=unknown

with open('Data/action_type.json', "w") as outfile:
    json.dump(action_type, outfile)

In [ ]:
cnt=0
for w,resp in mental.items():
    print(w)
    print(resp)
    print('\n') 
    cnt+=1
    if cnt>10:
        break